## Final Project Submission

Please fill out:
* Student name: Phillip Ojo
* Student pace: Part time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


In [2]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

I will combine all the dataset into one large one since my target variable as indepent of specific car model.

In [22]:
df1 = pd.read_csv('opel_corsa_01.csv', delimiter= ';')
df2 = pd.read_csv('opel_corsa_02.csv', delimiter= ';')
df3 = pd.read_csv('peugeot_207_01.csv', delimiter= ';')
df4 = pd.read_csv('peugeot_207_02.csv', delimiter= ';')

In [23]:
all_dfs = [df1, df2, df3, df4]
car_data = pd.concat(all_dfs)

# EDA

In [32]:
car_data

,AltitudeVariation,VehicleSpeedInstantaneous,VehicleSpeedAverage,VehicleSpeedVariance,VehicleSpeedVariation,LongitudinalAcceleration,EngineLoad,EngineCoolantTemperature,ManifoldAbsolutePressure,EngineRPM,MassAirFlow,IntakeAirTemperature,VerticalAcceleration,FuelConsumptionAverage,roadSurface,traffic,drivingStyle
0,NaN,0,NaN,NaN,NaN,"0,0156","25,49019623",64.0,100.0,801,"7,849999905",22.0,"-0,0078",NaN,SmoothCondition,LowCongestionCondition,EvenPaceStyle
1,NaN,0,NaN,NaN,NaN,"0,0156","25,49019623",64.0,100.0,803,"7,889999866",22.0,"-0,0156",NaN,SmoothCondition,LowCongestionCondition,EvenPaceStyle
2,NaN,NaN,NaN,NaN,NaN,"0,0273","25,88235283",64.0,100.0,800,"7,769999981",22.0,"-0,0273",NaN,SmoothCondition,LowCongestionCondition,EvenPaceStyle
3,NaN,NaN,NaN,NaN,NaN,"0,0391","25,88235283",64.0,100.0,798,"7,769999981",22.0,"-0,0273",NaN,SmoothCondition,LowCongestionCondition,EvenPaceStyle
4,NaN,0,NaN,NaN,NaN,"0,0469","25,88235283",65.0,100.0,798,"7,940000057",22.0,"-0,0312",NaN,SmoothCondition,LowCongestionCondition,EvenPaceStyle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4618,1,"28,79999924","28,55999908","57,19057079","3,600000381","-0,0292","25,88235283",81.0,115.0,"1755,5","20,46999931",25.0,"-0,1661","14,57800293",SmoothCondition,LowCongestionCondition,EvenPaceStyle
4619,"1,699996948","30,59999847","28,5299991","57,01026584","1,799999237","-0,0304","11,76470566",81.0,106.0,"736,5","17,73999977",25.0,"-0,1987","14,58564186",SmoothCondition,LowCongestionCondition,EvenPaceStyle
4620,"1,800003052","29,69999886","28,49999908","56,8830454","-0,899999619","-0,1684","98,03921509",81.0,106.0,1254,"9,520000458",24.0,"-0,1156","14,54729366",SmoothCondition,LowCongestionCondition,EvenPaceStyle
4621,"2,100006104","29,69999886","28,40999908","56,16090993",0,"-0,0644","79,60784149",80.0,112.0,1254,"14,90999985",23.0,"-0,076","14,54682827",SmoothCondition,LowCongestionCondition,EvenPaceStyle


In [33]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24957 entries, 0 to 4622
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   AltitudeVariation          24777 non-null  object 
 1   VehicleSpeedInstantaneous  24913 non-null  object 
 2   VehicleSpeedAverage        23775 non-null  object 
 3   VehicleSpeedVariance       23775 non-null  object 
 4   VehicleSpeedVariation      24769 non-null  object 
 5   LongitudinalAcceleration   24957 non-null  object 
 6   EngineLoad                 24952 non-null  object 
 7   EngineCoolantTemperature   24952 non-null  float64
 8   ManifoldAbsolutePressure   24952 non-null  float64
 9   EngineRPM                  24952 non-null  object 
 10  MassAirFlow                24952 non-null  object 
 11  IntakeAirTemperature       24952 non-null  float64
 12  VerticalAcceleration       24957 non-null  object 
 13  FuelConsumptionAverage     24671 non-null  obje

In [34]:
car_data['traffic'].value_counts()

LowCongestionCondition       18769
NormalCongestionCondition     3171
HighCongestionCondition       3017
Name: traffic, dtype: int64

In [35]:
car_data['roadSurface'].value_counts()

SmoothCondition         15242
UnevenCondition          6466
FullOfHolesCondition     3249
Name: roadSurface, dtype: int64

In [36]:
car_data['drivingStyle'].value_counts()

EvenPaceStyle      22089
AggressiveStyle     2868
Name: drivingStyle, dtype: int64